<a href="https://colab.research.google.com/github/Dimildizio/DS_course/blob/main/Neural_networks/Convolutions/Breast_cancer_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Breast cancer segmentation

## Installs & Imports

In [3]:
%%capture
#pip install catalyst

In [1]:
#import albumentations as albs
import os
import torch
import torch.nn as nn

from glob import glob
from torch import optim
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import ToTensor
from IPython import display

#from catalyst import dl, utils

## Download dataset

In [2]:
!curl -JLO 'https://www.dropbox.com/scl/fi/gs3kzp6b8k6faf667m5tt/breast-cancer-cells-segmentation.zip?rlkey=md3mzikpwrvnaluxnhms7r4zn'
!unzip breast-cancer-cells-segmentation.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   256    0   256    0     0    335      0 --:--:-- --:--:-- --:--:--   335
100   491    0   491    0     0    332      0 --:--:--  0:00:01 --:--:--   332
100 93.6M  100 93.6M    0     0  31.9M      0  0:00:02  0:00:02 --:--:--  104M


## EDA

In [5]:
!unzip breast-cancer-cells-segmentation.zip

Archive:  breast-cancer-cells-segmentation.zip
  inflating: Images/ytma10_010704_benign1_ccd.tif  
  inflating: Images/ytma10_010704_benign1_ccd.tif.xml  
  inflating: Images/ytma10_010704_benign2_ccd.tif  
  inflating: Images/ytma10_010704_benign2_ccd.tif.xml  
  inflating: Images/ytma10_010704_benign3_ccd.tif  
  inflating: Images/ytma10_010704_benign3_ccd.tif.xml  
  inflating: Images/ytma10_010704_malignant1_ccd.tif  
  inflating: Images/ytma10_010704_malignant1_ccd.tif.xml  
  inflating: Images/ytma10_010704_malignant2_ccd.tif  
  inflating: Images/ytma10_010704_malignant2_ccd.tif.xml  
  inflating: Images/ytma10_010704_malignant3_ccd.tif  
  inflating: Images/ytma10_010704_malignant3_ccd.tif.xml  
  inflating: Images/ytma12_010804_benign1_ccd.tif  
  inflating: Images/ytma12_010804_benign1_ccd.tif.xml  
  inflating: Images/ytma12_010804_benign2_ccd.tif  
  inflating: Images/ytma12_010804_benign2_ccd.tif.xml  
  inflating: Images/ytma12_010804_benign3_ccd.tif  
  inflating: Images

In [16]:
imgs_paths = glob(os.path.join("/content/Images", "*.tif"))

In [15]:
print(os.getcwd())

/content


In [19]:
masks_paths = [os.path.join("/content/Masks", i.rsplit("/",1)[-1].split("_ccd")[0]+".TIF") for i in imgs_paths]

In [21]:
print(masks_paths)

['/content/Masks/ytma49_072303_malignant2.TIF', '/content/Masks/ytma12_010804_malignant1.TIF', '/content/Masks/ytma49_111303_malignant3.TIF', '/content/Masks/ytma49_042203_malignant3.TIF', '/content/Masks/ytma23_022103_malignant3.TIF', '/content/Masks/ytma49_042003_benign1.TIF', '/content/Masks/ytma10_010704_malignant2.TIF', '/content/Masks/ytma49_111003_benign2.TIF', '/content/Masks/ytma49_072303_malignant1.TIF', '/content/Masks/ytma55_030603_benign3.TIF', '/content/Masks/ytma49_042003_benign2.TIF', '/content/Masks/ytma23_022103_malignant2.TIF', '/content/Masks/ytma49_111303_malignant1.TIF', '/content/Masks/ytma49_042203_malignant2.TIF', '/content/Masks/ytma49_042003_benign3.TIF', '/content/Masks/ytma49_042203_benign2.TIF', '/content/Masks/ytma49_042003_malignant1.TIF', '/content/Masks/ytma49_042403_malignant3.TIF', '/content/Masks/ytma23_022103_benign2.TIF', '/content/Masks/ytma10_010704_malignant3.TIF', '/content/Masks/ytma49_042403_benign3.TIF', '/content/Masks/ytma12_010804_malign

## Write Dataloaders and transform data